## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2026-02-20-19-07-53 +0000,nyt,The Supreme Court’s Declaration of Independence,https://www.nytimes.com/2026/02/20/us/politics...
1,2026-02-20-19-06-16 +0000,wapo,"Trump imposes new 10% global tariff, replacing...",https://www.washingtonpost.com
2,2026-02-20-19-06-16 +0000,wapo,"Live updates: Trump imposes new tariffs, denou...",https://www.washingtonpost.com/national-securi...
3,2026-02-20-19-05-15 +0000,nyt,"Trump and Indonesia Sign Trade Deal, Locking I...",https://www.nytimes.com/2026/02/20/business/tr...
4,2026-02-20-19-03-05 +0000,bbc,Welcome to Australia's hottest beach event - n...,https://www.bbc.com/news/articles/crrx792qk81o...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2460/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
4,trump,82
15,tariffs,31
1,court,29
0,supreme,26
40,andrew,25


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
29,2026-02-20-18-15-00 +0000,wsj,The Supreme Court decision to strike down Pres...,https://www.wsj.com/economy/trade/trump-tariff...,237
2,2026-02-20-19-06-16 +0000,wapo,"Live updates: Trump imposes new tariffs, denou...",https://www.washingtonpost.com/national-securi...,220
31,2026-02-20-18-10-17 +0000,startribune,Brown: As Supreme Court upends Trump’s disrupt...,https://www.startribune.com/supreme-court-just...,197
73,2026-02-20-17-20-00 +0000,wsj,The Supreme Court struck down a swath of Presi...,https://www.wsj.com/economy/trade/courts-tarif...,196
71,2026-02-20-17-23-09 +0000,nypost,Democrats giddy over Trump Supreme Court tarif...,https://nypost.com/2026/02/20/us-news/democrat...,195


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
29,237,2026-02-20-18-15-00 +0000,wsj,The Supreme Court decision to strike down Pres...,https://www.wsj.com/economy/trade/trump-tariff...
326,124,2026-02-19-22-28-26 +0000,nypost,Bombshell Jeffrey Epstein files that led to ex...,https://nypost.com/2026/02/19/world-news/these...
127,71,2026-02-20-15-21-20 +0000,nypost,Hundreds of firms poised to be banned from sna...,https://nypost.com/2026/02/20/business/hundred...
168,63,2026-02-20-13-00-00 +0000,wsj,After President Trump’s first year back in off...,https://www.wsj.com/economy/trade/tariffs-comm...
294,61,2026-02-20-00-00-00 +0000,wsj,The husband of embattled Labor Secretary Lori ...,https://www.wsj.com/politics/policy/labor-secr...
71,54,2026-02-20-17-23-09 +0000,nypost,Democrats giddy over Trump Supreme Court tarif...,https://nypost.com/2026/02/20/us-news/democrat...
124,53,2026-02-20-15-29-29 +0000,wapo,Trump talks peace in the Middle East as he rea...,https://www.washingtonpost.com/national-securi...
184,41,2026-02-20-11-49-00 +0000,wsj,What on Earth? President Trump alleged that fo...,https://www.wsj.com/politics/on-trump-obama-al...
305,38,2026-02-19-23-31-49 +0000,nypost,Calif. gov candidate Tom Steyer privately told...,https://nypost.com/2026/02/19/us-news/tom-stey...
375,38,2026-02-19-19-46-16 +0000,nypost,Texas AG sues drone giant for Chinese Communis...,https://nypost.com/2026/02/19/business/texas-a...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
